In [74]:
import json
import numpy as np
import tensorflow as tf

In [75]:
# Define user and item data
# user_data = [
#     {
#         "user_id": 1,
#         "interests": ["hành động", "adventure"],
#         "liked": [102, 103]
#     },
#     {
#         "user_id": 2,
#         "interests": ["comedy", "romance"],
#         "liked": [102, 105]
#     },
#     {
#         "user_id": 3,
#         "interests": ["science fiction"],
#         "liked": [104]
#     }
# ]

# item_data = [
#     {
#         "item_id": 101,
#         "genres": ["hành động", "adventure"]
#     },
#     {
#         "item_id": 102,
#         "genres": ["comedy", "romance"]
#     },
#     {
#         "item_id": 103,
#         "genres": ["hành động", "adventure"]
#     },
#     {
#         "item_id": 104,
#         "genres": ["science fiction"]
#     },
#     {
#         "item_id": 105,
#         "genres": ["comedy"]
#     }
# ]


In [76]:
# Load user interests from JSON (replace 'your_interests.json' with your JSON file)
with open('../data/users.json', 'r', encoding='utf-8') as json_file:
    user_data = json.load(json_file)

# Load item data from JSON (replace 'your_items.json' with your JSON file)
with open('../data/campaigns.json', 'r', encoding='utf-8') as json_file:
    item_data = json.load(json_file)

In [77]:
# Create mappings for user and item IDs
user_ids = [user["user_id"] for user in user_data]
item_ids = [item["item_id"] for item in item_data]

user_id_to_index = {user_id: index for index, user_id in enumerate(user_ids)}
item_id_to_index = {item_id: index for index, item_id in enumerate(item_ids)}

print(user_id_to_index)


{1: 0, 2: 1, 3: 2}


In [78]:
# Create user-item matrix
num_users = len(user_ids)
num_items = len(item_ids)

user_item_matrix = np.zeros((num_users, num_items), dtype=np.float32)

# Fill in the user-item matrix with 1.0 for liked items
for user in user_data:
    user_index = user_id_to_index[user["user_id"]]
    for item_id in user["liked"]:
        item_index = item_id_to_index[item_id]
        user_item_matrix[user_index, item_index] = 1.0

print(user_item_matrix)

[[0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [79]:
# Define the matrix factorization model
latent_dim = 10

# Define input layers for user and item IDs
user_input = tf.keras.layers.Input(shape=(1,))
item_input = tf.keras.layers.Input(shape=(1,))

# Create embeddings for users and items
user_embedding = tf.keras.layers.Embedding(num_users, latent_dim)(user_input)
item_embedding = tf.keras.layers.Embedding(num_items, latent_dim)(item_input)

# Create embeddings for user and item biases
user_bias = tf.keras.layers.Embedding(num_users, 1)(user_input)
item_bias = tf.keras.layers.Embedding(num_items, 1)(item_input)

# Calculate dot product of user and item embeddings
dot_product = tf.keras.layers.Dot(axes=2)([user_embedding, item_embedding])

# Add user and item biases to the dot product
dot_product = tf.keras.layers.Add()([dot_product, user_bias, item_bias])

# Create the recommendation model
model = tf.keras.Model(inputs=[user_input, item_input], outputs=dot_product)

print(user_input)

KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name='input_15'), name='input_15', description="created by layer 'input_15'")


In [80]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

In [81]:
# Train the model
user_indices = [user_id_to_index[user["user_id"]] for user in user_data for _ in user["liked"]]
item_indices = [item_id_to_index[item_id] for user in user_data for item_id in user["liked"]]
ratings = [1.0] * len(user_indices)

history = model.fit(
    [np.array(user_indices), np.array(item_indices)],
    np.array(ratings),
    epochs=50,
    verbose=1,
    validation_split=0.2
)

print(user_indices)


Epoch 1/50


1/1 [==============================] - 1s 739ms/step - loss: 0.9822 - val_loss: 1.0328
Epoch 2/50
1/1 [==============================] - 0s 28ms/step - loss: 0.9775 - val_loss: 1.0328
Epoch 3/50
1/1 [==============================] - 0s 28ms/step - loss: 0.9727 - val_loss: 1.0328
Epoch 4/50
1/1 [==============================] - 0s 27ms/step - loss: 0.9679 - val_loss: 1.0328
Epoch 5/50
1/1 [==============================] - 0s 28ms/step - loss: 0.9632 - val_loss: 1.0328
Epoch 6/50
1/1 [==============================] - 0s 27ms/step - loss: 0.9584 - val_loss: 1.0328
Epoch 7/50
1/1 [==============================] - 0s 28ms/step - loss: 0.9537 - val_loss: 1.0328
Epoch 8/50
1/1 [==============================] - 0s 28ms/step - loss: 0.9490 - val_loss: 1.0328
Epoch 9/50
1/1 [==============================] - 0s 27ms/step - loss: 0.9442 - val_loss: 1.0328
Epoch 10/50
1/1 [==============================] - 0s 28ms/step - loss: 0.9395 - val_loss: 1.0328
Epoch 11/50
1/1 [======================

In [82]:
print(model.summary())

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_15 (InputLayer)       [(None, 1)]                  0         []                            
                                                                                                  
 input_16 (InputLayer)       [(None, 1)]                  0         []                            
                                                                                                  
 embedding_28 (Embedding)    (None, 1, 10)                30        ['input_15[0][0]']            
                                                                                                  
 embedding_29 (Embedding)    (None, 1, 10)                90        ['input_16[0][0]']            
                                                                                            

In [83]:
# Make recommendations for a specific user (e.g., user_id = 1)
user_id_to_recommend = 1
user_index_to_recommend = user_id_to_index[user_id_to_recommend]
user_embedding_weights = model.layers[2].get_weights()[0]
# Access item embedding weights
item_embedding_weights = model.layers[3].get_weights()[0]

# # Get the top N recommended items

top_n = 5
# # Calculate the number of items
num_items = len(item_ids)

# # Ensure top_n does not exceed the number of items
top_n = min(top_n, num_items)

# Calculate predicted ratings for all items for the user
user_ratings = np.dot(user_embedding_weights[user_index_to_recommend], item_embedding_weights.T)

# Get the indices of items sorted by user ratings in descending order
sorted_item_indices = np.argsort(user_ratings)[::-1]

# Create a set of items that the user has liked for efficient lookup
liked_items = set(user_data[user_index_to_recommend]["liked"])

# Debug print to check liked_items
print("Liked Items:", liked_items)
# Filter out items that the user has already liked
# recommended_item_indices = [item_ids[i] for i in sorted_item_indices if item_ids[i] not in liked_items][:top_n]

recommended_item_indices = [item_ids[i] for i in sorted_item_indices if item_ids[i] not in liked_items]

top_item_indices = recommended_item_indices[:top_n]

recommended_item_ids = [i for i in top_item_indices ]

# Retrieve the actual item data for the recommended items
# recommended_items_data = [item for item in item_data if item["item_id"] in recommended_item_indices]

# Get the corresponding item IDs for the top N recommended items
# recommended_item_ids = [i for i in top_item_indices ]

# print(f"Top {top_n} recommended items for user {user_id_to_recommend}:")
# print(recommended_item_indices)

print(f"Top {top_n} recommended items for user {user_id_to_recommend}: {recommended_item_ids}")



Liked Items: {102, 103}
Top 5 recommended items for user 1: [105, 109, 106, 108, 107]
